In [1]:
import pandas as pd 
import numpy as np

df = pd.read_csv('data/feedback_clientes_v2.csv')

In [2]:
df.head()

,Feedback_ID,Transaccion_ID,Rating_Producto,Rating_Logistica,Comentario_Texto,Recomienda_Marca,Ticket_Soporte_Abierto,Edad_Cliente,Satisfaccion_NPS
0,FB-8000,TRX-17461,99,4,NaN,NaN,Sí,195,-17.5
1,FB-8001,TRX-17755,4,5,---,Maybe,Sí,59,-41.7
2,FB-8002,TRX-10534,3,4,No volvería,Maybe,0,84,-36.4
3,FB-8003,TRX-12569,2,3,---,NaN,Sí,20,7.4
4,FB-8004,TRX-19159,4,2,Dañado,SI,No,83,61.0


In [3]:
df['Feedback_ID'].isna().sum()

np.int64(0)

In [4]:
df['Transaccion_ID'].isna().sum()

np.int64(0)

In [5]:
df['Rating_Producto'].isna().sum()

np.int64(0)

In [6]:
df['Rating_Logistica'].isna().sum()

np.int64(0)

In [7]:
df['Comentario_Texto'].isna().sum()

np.int64(657)

In [8]:
df['Recomienda_Marca'].isna().sum()

np.int64(1119)

In [9]:
df['Ticket_Soporte_Abierto'].isna().sum()

np.int64(0)

In [10]:
df['Edad_Cliente'].isna().sum()

np.int64(0)

In [11]:
df['Satisfaccion_NPS'].isna().sum()

np.int64(0)

In [12]:
#Revisamos primero las columnas categóricas
categorical_cols = ['Recomienda_Marca', 'Ticket_Soporte_Abierto', 'Comentario_Texto']
for col in categorical_cols:
    print(f"Valores únicos en la columna {col}: {df[col].unique()}")


Valores únicos en la columna Recomienda_Marca: [nan 'Maybe' 'SI' 'NO']
Valores únicos en la columna Ticket_Soporte_Abierto: ['Sí' '0' 'No' '1']
Valores únicos en la columna Comentario_Texto: [nan '---' 'No volvería' 'Dañado' 'Precio justo' 'Lento' 'Excelente']


In [13]:
df['Ticket_Soporte_Abierto'] = df['Ticket_Soporte_Abierto'].replace({'1': 'Sí', '0': 'No'})
df['Comentario_Texto'] = df['Comentario_Texto'].replace({'---': np.nan})

In [14]:
df['Recomienda_Marca'].isna().sum()

np.int64(1119)

In [15]:
df['Recomienda_Marca'] = df['Recomienda_Marca'].str.upper()

In [16]:
df[['Recomienda_Marca', 'Comentario_Texto']].value_counts(dropna=False)

Recomienda_Marca  Comentario_Texto
SI                NaN                 353
MAYBE             NaN                 322
NO                NaN                 312
NaN               NaN                 301
NO                Dañado              192
                  Lento               179
SI                Precio justo        177
NaN               Excelente           176
NO                Excelente           173
MAYBE             Lento               169
SI                Excelente           169
NaN               Dañado              169
                  No volvería         168
                  Lento               165
MAYBE             No volvería         159
                  Excelente           159
SI                Dañado              157
                  Lento               155
                  No volvería         151
NO                No volvería         146
                  Precio justo        140
NaN               Precio justo        140
MAYBE             Precio justo        139

In [17]:
df[['Rating_Producto', 'Recomienda_Marca']].value_counts(normalize=True, dropna=False) * 100

Rating_Producto  Recomienda_Marca
5                SI                  5.666667
                 NO                  5.466667
1                NaN                 5.422222
3                SI                  5.422222
1                NO                  5.377778
3                NaN                 5.222222
2                NaN                 5.022222
4                SI                  5.022222
5                MAYBE               5.000000
2                NO                  4.977778
1                MAYBE               4.955556
3                MAYBE               4.866667
2                SI                  4.800000
4                NO                  4.800000
1                SI                  4.733333
2                MAYBE               4.666667
5                NaN                 4.577778
3                NO                  4.555556
4                NaN                 4.422222
                 MAYBE               4.355556
99               NO                  0.200000


In [18]:
df['Comentario_Texto'].fillna(df['Comentario_Texto'].mode()[0], inplace=True)
df['Recomienda_Marca'].fillna(df['Recomienda_Marca'].mode()[0], inplace=True)

In [19]:
# Función para manejar outliers en Rating_Producto
def manejar_outliers_rating_producto(df, medida='Mediana'):
    """
    Detecta y reemplaza outliers en la columna Rating_Producto.
    
    Parámetros:
    -----------
    df : DataFrame
        Dataframe que contiene la columna Rating_Producto
    medida : str
        Medida para reemplazar outliers: 'Moda', 'Mediana' o 'Media'
        
    Retorna:
    --------
    DataFrame : Dataframe con outliers reemplazados
    """
    
    df_copy = df.copy()
    columna = 'Rating_Producto'
    
    # Detectar outliers usando IQR
    Q1 = df_copy[columna].quantile(0.25)
    Q3 = df_copy[columna].quantile(0.75)
    IQR = Q3 - Q1
    
    limite_inferior = Q1 - 1.5 * IQR
    limite_superior = Q3 + 1.5 * IQR

    if limite_inferior < 1:
        limite_inferior = 0
    
    # Identificar outliers
    outliers_mask = (df_copy[columna] < limite_inferior) | (df_copy[columna] > limite_superior)
    num_outliers = outliers_mask.sum()
    
    # Seleccionar la medida de reemplazo
    if medida.lower() == 'moda':
        valor_reemplazo = df_copy[columna].mode()[0]
    elif medida.lower() == 'mediana':
        valor_reemplazo = df_copy[columna].median()
    elif medida.lower() == 'media':
        valor_reemplazo = df_copy[columna].mean()
    else:
        raise ValueError("La medida debe ser 'Moda', 'Mediana' o 'Media'")
    
    # Reemplazar outliers
    df_copy.loc[outliers_mask, columna] = valor_reemplazo
    
    print(f"Outliers detectados: {num_outliers}")
    print(f"Límite inferior: {limite_inferior:.2f}")
    print(f"Límite superior: {limite_superior:.2f}")
    print(f"Valor de reemplazo ({medida}): {valor_reemplazo:.2f}")
    print(f"Outliers reemplazados exitosamente")
    
    return df_copy

# Ejemplo de uso:
df = manejar_outliers_rating_producto(df, medida='Mediana')

Outliers detectados: 30
Límite inferior: 0.00
Límite superior: 7.00
Valor de reemplazo (Mediana): 3.00
Outliers reemplazados exitosamente


In [20]:
df['Rating_Producto'].describe()

count    4500.000000
mean        2.995778
std         1.424465
min         1.000000
25%         2.000000
50%         3.000000
75%         4.000000
max         5.000000
Name: Rating_Producto, dtype: float64

In [21]:
df[df['Recomienda_Marca'] == 'MAYBE'][['Recomienda_Marca', 'Rating_Producto']].value_counts()

Recomienda_Marca  Rating_Producto
MAYBE             5                  225
                  1                  223
                  3                  223
                  2                  210
                  4                  196
Name: count, dtype: int64

In [22]:
df.describe()

,Rating_Producto,Rating_Logistica,Edad_Cliente,Satisfaccion_NPS
count,4500.000000,4500.000000,4500.000000,4500.000000
mean,2.995778,3.005111,51.172889,0.303089
std,1.424465,1.418520,21.830784,57.184315
min,1.000000,1.000000,18.000000,-99.800000
25%,2.000000,2.000000,34.000000,-49.200000
50%,3.000000,3.000000,50.000000,1.100000
75%,4.000000,4.000000,67.000000,49.525000
max,5.000000,5.000000,195.000000,99.900000


In [23]:
df['Edad_Cliente'].describe()

count    4500.000000
mean       51.172889
std        21.830784
min        18.000000
25%        34.000000
50%        50.000000
75%        67.000000
max       195.000000
Name: Edad_Cliente, dtype: float64

In [24]:
# Función para manejar outliers en Rating_Producto
def manejar_outliers_edad_cliente(df, medida='Mediana'):
    """
    Detecta y reemplaza outliers en la columna Edad_Cliente.
    
    Parámetros:
    -----------
    df : DataFrame
        Dataframe que contiene la columna Edad_Cliente
    medida : str
        Medida para reemplazar outliers: 'Moda', 'Mediana' o 'Media'
        
    Retorna:
    --------
    DataFrame : Dataframe con outliers reemplazados
    """
    
    df_copy = df.copy()
    columna = 'Edad_Cliente'
    
    # Detectar outliers usando IQR
    Q1 = df_copy[columna].quantile(0.25)
    Q3 = df_copy[columna].quantile(0.75)
    IQR = Q3 - Q1
    
    limite_inferior = Q1 - 1.5 * IQR
    limite_superior = Q3 + 1.5 * IQR

    if limite_inferior < 1:
        limite_inferior = 0
    
    # Identificar outliers
    outliers_mask = (df_copy[columna] < limite_inferior) | (df_copy[columna] > limite_superior)
    num_outliers = outliers_mask.sum()
    
    # Seleccionar la medida de reemplazo
    if medida.lower() == 'moda':
        valor_reemplazo = df_copy[columna].mode()[0]
    elif medida.lower() == 'mediana':
        valor_reemplazo = df_copy[columna].median()
    elif medida.lower() == 'media':
        valor_reemplazo = df_copy[columna].mean()
    else:
        raise ValueError("La medida debe ser 'Moda', 'Mediana' o 'Media'")
    
    # Reemplazar outliers
    df_copy.loc[outliers_mask, columna] = valor_reemplazo
    
    print(f"Outliers detectados: {num_outliers}")
    print(f"Límite inferior: {limite_inferior:.2f}")
    print(f"Límite superior: {limite_superior:.2f}")
    print(f"Valor de reemplazo ({medida}): {valor_reemplazo:.2f}")
    print(f"Outliers reemplazados exitosamente")
    
    return df_copy

# Ejemplo de uso:
df = manejar_outliers_edad_cliente(df, medida='Mediana')


Outliers detectados: 23
Límite inferior: 0.00
Límite superior: 116.50
Valor de reemplazo (Mediana): 50.00
Outliers reemplazados exitosamente


In [25]:
df['Edad_Cliente'].describe()

count    4500.000000
mean       50.431778
std        19.242859
min        18.000000
25%        34.000000
50%        50.000000
75%        67.000000
max        84.000000
Name: Edad_Cliente, dtype: float64

In [26]:
df['Transaccion_ID'].describe()

count          4500
unique         3623
top       TRX-19467
freq              4
Name: Transaccion_ID, dtype: object

In [27]:
df['Ticket_Soporte_Abierto'].value_counts()

Ticket_Soporte_Abierto
Sí    2298
No    2202
Name: count, dtype: int64

In [29]:
df.to_csv('data/feedback_clientes_limpio.csv', index=False)